In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as Layer
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

import importlib
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = "{:,.2f}".format

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
# logical_gpus = tf.config.experimental.list_logical_devices('GPU')
# print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
print("Tensorflow version: ", tf.__version__)
print("Tensorflow GPU detection: ", tf.config.list_physical_devices('GPU'))

Tensorflow version:  2.5.0
Tensorflow GPU detection:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
DATA_PATH = '../data/G2Net/g2net-gravitational-wave-detection/'
MODEL_PATH = '../data/G2Net/models'
TB_LOG_PATH = '../data/G2Net/tb_logs'
AUTOTUNE = tf.data.AUTOTUNE

SPLIT_RATIO = .9
RANDOM_STATE = 42

IMG_WIDTH = 69
IMG_HEIGHT = 193


In [4]:
train = tf.data.experimental.load(os.path.join(DATA_PATH, 'ds_train'))
valid = tf.data.experimental.load(os.path.join(DATA_PATH, 'ds_valid'))

In [5]:
def prep(img, label):
    ret = preprocess_input(img)
    return ret,label

In [6]:
BATCH_SIZE = 16

# train_data = train.take(250000).map(prep).batch(BATCH_SIZE).prefetch(buffer_size=64)
# valid_data = valid.take(25000).map(prep).cache().batch(BATCH_SIZE).prefetch(buffer_size=64)
train_data = train.take(100000).batch(BATCH_SIZE).prefetch(buffer_size=64)
valid_data = valid.take(10000).cache().batch(BATCH_SIZE).prefetch(buffer_size=64)

In [7]:
print(train_data.cardinality)

<bound method DatasetV2.cardinality of <PrefetchDataset shapes: (<unknown>, (None,)), types: (tf.float32, tf.float32)>>


In [8]:
from tensorflow.keras.applications.efficientnet import *
from tensorflow.keras.applications.efficientnet import preprocess_input

base_model = EfficientNetB0(
    include_top=False,
    input_shape=(IMG_WIDTH, IMG_HEIGHT,3),
    weights='imagenet',
    drop_connect_rate=0.3)

# base_model = ResNet50V2(
#     weights='imagenet',
# #     weights=None,
#     include_top=False,
#     input_shape=(IMG_WIDTH, IMG_HEIGHT,3),
#     pooling='avg')
# base_model.trainable=True
# base_model.trainable=False


# fine_tune_at = 171
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable =  False

In [9]:
metrics = [tf.keras.metrics.AUC(name='AUC'),
        tf.keras.metrics.BinaryAccuracy(name='Accuracy'),]

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=2e-4,
    decay_steps=4000,
    decay_rate=0.9)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(TB_LOG_PATH,'effb0_base_1e4'),
    update_freq=64,
    )



callbacks= [
    tb_callback,
]

def create_model(alpha):
    model = tf.keras.Sequential([
        Layer.InputLayer(input_shape=(IMG_WIDTH, IMG_HEIGHT,1)),
#         Layer.experimental.preprocessing.Rescaling(255),
        Layer.Conv2D(3,3,padding='same', use_bias=False),
        Layer.BatchNormalization(),
#         Layer.Lambda(preprocess_input),
#         Layer.Dropout(.3),
        base_model,
        Layer.GlobalAveragePooling2D(name="avg_pool"),
        Layer.BatchNormalization(),
        Layer.Dropout(.2, name="top_dropout"),
        #         Layer.Dropout(.3),
#         Layer.Dense(32,activation='relu'),
        Layer.Dense(1, activation='sigmoid')])

#     Layer.Dense(256),
#         Layer.LeakyReLU(alpha=alpha),
#         Layer.Dense(256),
#         Layer.LeakyReLU(alpha=alpha),
# #         Layer.Dense(8),
# #         Layer.Dropout(.3),
# #         Layer.LeakyReLU(alpha=alpha),
#         Layer.Dense(1, activation='sigmoid'),
#     ])
    return model

In [10]:
model = create_model(.3)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 193, 3)        27        
_________________________________________________________________
batch_normalization (BatchNo (None, 69, 193, 3)        12        
_________________________________________________________________
efficientnetb0 (Functional)  (None, 3, 7, 1280)        4049571   
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 1280)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1280)              5120      
_________________________________________________________________
top_dropout (Dropout)        (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1

In [11]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  237


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    ),
    loss=tf.keras.losses.BinaryCrossentropy(), 
    metrics=metrics)

In [ ]:
model.fit(
    train_data,
    epochs=2,
    validation_data=valid_data,
    callbacks = callbacks,
    )

c:\users\vince\miniconda3\envs\ml38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/2
6250/6250 [==============================] - 729s 114ms/step - loss: 0.5747 - AUC: 0.7618 - Accuracy: 0.7017 - val_loss: 0.5086 - val_AUC: 0.8261 - val_Accuracy: 0.7441
Epoch 2/2
3619/6250 [================>.............] - ETA: 4:07 - loss: 0.5143 - AUC: 0.8085 - Accuracy: 0.7431

In [27]:
model.save(os.path.join(MODEL_PATH,'effb0tl_1e3'))

c:\users\vince\miniconda3\envs\ml38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ../data/G2Net/models\effb0tl_1e3\assets


In [23]:
pred = model.predict(valid_data.take(100))

In [24]:
print(pred)

[[0.18]
 [0.14]
 [0.26]
 ...
 [0.24]
 [0.29]
 [0.1 ]]


In [ ]:
for data, label in valid_data.skip(100).take(1):
    print(data)